In [1]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

In [24]:
def ppmi(ppmi_matrix, word_dict, x, y):
    if x not in word_dict.keys():
        return 0
    elif y not in word_dict.keys():
        return 0

    x_id = word_dict[x]
    y_id = word_dict[y]
    return ppmi_matrix[x_id, y_id]

In [2]:
file_path = "../../corpus/collocation/ppmi_ntt1.npy"
ppmi_matrix = np.load(file_path)

In [28]:
dictname = "../../corpus/collocation/word_dict/ppmi_word_dict_ntt1.json"
with open(dictname, "r") as f:
    word_dict = json.load(f)

In [9]:
path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [12]:
def make_Xy(convs, N=7):
    errors = ["Topic transition error"]
    print(errors)
    X = []
    y = []
    for conv in convs:
        dialogue = [""]*N
        for i, ut in enumerate( conv ) :
            # ユーザ発話駆動
            dialogue.append(clean_text( ut.utt) )
            if ut.is_system():
                X.append( dialogue[-N:] )
                    # X.append(dialogue[-N:])
                if ut.is_error_included(errors) :
                    y.append(1)
                else:
                    y.append(0)
        
    return X, y

In [21]:
N = 4
X_str, y = make_Xy(convs, N=4)

['Topic transition error']


In [14]:
len( X_str ) 

2200

In [17]:
# idf 辞書
border_005 = 5.340630805533431
idf_path = "../../corpus/wiki/idf_wiki_v2.json"
with open(idf_path, "r") as f:
    idf_dict = json.load(f)

In [19]:
toyoshima_set = set("NOUN PROPN VERB ADJ".split())

def filtering(doc, filter_set):
    left = []
    for token in doc:
        if token.pos_ in filter_set:
            left.append(token.lemma_)
    return left if len(left)>0 else ["[NONE]"]

In [31]:
# 2つの発話間の単語の相性を計算
def colocate_rate(utt1, utt2):
    left_1 = filtering( nlp(utt1) , toyoshima_set)
    left_2 = filtering( nlp(utt2) , toyoshima_set)
    
    # 名詞がないよ
    if left_1[0] == "[NONE]" or  left_2[0] == "[NONE]":
        return 5

    # print(left_1, left_2)
    rate = 0
    for lx in left_1:
        for ly in left_2:
            rate += ppmi(ppmi_matrix, word_dict, lx, ly)
    rate = rate/(len(left_1)+len(left_2))
    return rate
